<center><h1>Chang_Yi_HW3</h1></center>
<br>
<br>

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

#### Package imports

In [1]:
import pandas as pd
import os
import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

#### Get the AReM Data Set

In [2]:
activities = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
data_directory = '../data/AReM/'

activity_dfs = []

for activity in activities:
    directory_path = os.path.join(data_directory, activity)
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    data_frames = []

    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path, skiprows=4)
        data_frames.append(df)

    activity_df = pd.concat(data_frames, ignore_index=True)

    activity_df['activity'] = activity

    activity_dfs.append(activity_df)

combined_df = pd.concat(activity_dfs, ignore_index=True)
combined_df

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,0,47.33,0.47,28.00,0.82,34.25,1.30,bending1
1,250,47.50,0.50,29.00,1.22,36.00,0.00,bending1
2,500,47.50,0.50,30.00,0.00,36.75,0.83,bending1
3,750,47.00,0.71,28.00,1.22,37.00,1.22,bending1
4,1000,45.00,0.71,24.25,1.79,36.00,0.00,bending1
...,...,...,...,...,...,...,...,...
42234,118750,31.50,1.66,12.50,3.20,14.25,4.44,walking
42235,119000,27.33,1.25,11.33,0.94,20.00,4.00,walking
42236,119250,37.80,7.68,14.20,2.48,17.25,0.83,walking
42237,119500,33.75,1.30,15.75,5.21,16.50,2.69,walking


### (b) Test and Train Data

In [5]:
test_data_rows = {
    'bending1': 960,   # 480 * 2
    'bending2': 960,
    'cycling': 1440,   # 480 * 3
    'lying': 1440,
    'sitting': 1440,
    'standing': 1440,
    'walking': 1440
}

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for activity, rows_to_keep in test_data_rows.items():
    test_data = combined_df[combined_df['activity'] == activity].head(rows_to_keep)
    
    # Corrected calculation of training data
    train_data = combined_df[combined_df['activity'] == activity].tail(len(combined_df[combined_df['activity'] == activity]) - rows_to_keep)
    
    test_df = pd.concat([test_df, test_data], ignore_index=True)
    train_df = pd.concat([train_df, train_data], ignore_index=True)

### (c) Feature Extraction

#### i. Research

Mean, Median, Variance, Standard Deviation, Skewness, Kurtosis
Minimum, Maximum, Range

#### ii. Extraction

In [10]:
directories = ['../data/AReM/bending1', '../data/AReM/bending2', '../data/AReM/cycling', '../data/AReM/lying',
               '../data/AReM/sitting', '../data/AReM/standing', '../data/AReM/walking']

# This part is done to ensure the csv files are in order
# def rename_files_with_zeros(directory):
#     csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    
#     for csv_file in csv_files:
#         file_path = os.path.join(directory, csv_file)
#         file_name, file_extension = os.path.splitext(csv_file)
#         prefix, number = file_name.split('dataset')
#         new_file_name = f'dataset{number.zfill(2)}{file_extension}'
#         os.rename(file_path, os.path.join(directory, new_file_name))

# for directory in directories:
#     rename_files_with_zeros(directory)

features_df = pd.DataFrame() 
features_to_extract = ['min', 'max', 'mean', 'median', 'std', 'Q1', 'Q3']

for directory in directories:
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    csv_files.sort()

    directory_features_df = pd.DataFrame() 

    for csv_file in csv_files:
        file_path = os.path.join(directory, csv_file)
        
        df = pd.read_csv(file_path, skiprows=4, usecols=[1, 2, 3, 4, 5, 6])

        instance_features = pd.DataFrame()
        
        for column in df.columns:
            series = df[column]
            instance_features[f'{features_to_extract[0]}_{column}'] = [np.min(series)]
            instance_features[f'{features_to_extract[1]}_{column}'] = [np.max(series)]
            instance_features[f'{features_to_extract[2]}_{column}'] = [np.mean(series)]
            instance_features[f'{features_to_extract[3]}_{column}'] = [np.median(series)]
            instance_features[f'{features_to_extract[4]}_{column}'] = [np.std(series)]
            instance_features[f'{features_to_extract[5]}_{column}'] = [np.percentile(series, 25)]
            instance_features[f'{features_to_extract[6]}_{column}'] = [np.percentile(series, 75)]

        directory_features_df = pd.concat([directory_features_df, instance_features], ignore_index=True)
    
    features_df = pd.concat([features_df, directory_features_df], ignore_index=True)

features_df

,min_avg_rss12,max_avg_rss12,mean_avg_rss12,median_avg_rss12,std_avg_rss12,Q1_avg_rss12,Q3_avg_rss12,min_var_rss12,max_var_rss12,mean_var_rss12,...,std_avg_rss23,Q1_avg_rss23,Q3_avg_rss23,min_var_rss23,max_var_rss23,mean_var_rss23,median_var_rss23,std_var_rss23,Q1_var_rss23,Q3_var_rss23
0,37.25,45.00,40.624792,40.50,1.475428,39.25,42.0000,0.0,1.30,0.358604,...,2.186168,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582308,0.0000,1.3000
1,38.00,45.67,42.812812,42.50,1.434054,42.00,43.6700,0.0,1.22,0.372437,...,1.993175,32.0000,34.50,0.00,3.11,0.571083,0.43,0.600383,0.0000,1.3000
2,35.00,47.40,43.954500,44.33,1.557210,43.00,45.0000,0.0,1.70,0.426250,...,1.997520,35.3625,36.50,0.00,1.79,0.493292,0.43,0.512971,0.0000,0.9400
3,33.00,47.75,42.179812,43.50,3.666840,39.15,45.0000,0.0,3.00,0.696042,...,3.845436,30.4575,36.33,0.00,2.18,0.613521,0.50,0.523771,0.0000,1.0000
4,33.00,45.75,41.678063,41.75,2.241152,41.33,42.7500,0.0,2.83,0.535979,...,2.408514,28.4575,31.25,0.00,1.79,0.383292,0.43,0.388759,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,19.50,45.33,33.586875,34.25,4.646088,30.25,37.0000,0.0,14.67,4.576562,...,3.280561,13.7300,18.25,0.00,8.32,3.259729,3.11,1.638534,2.0500,4.3225
84,19.75,45.50,34.322750,35.25,4.747524,31.00,38.0000,0.0,13.47,4.456333,...,3.116605,13.5000,17.75,0.00,9.67,3.432563,3.20,1.730921,2.1575,4.5650
85,19.50,46.00,34.546229,35.25,4.837247,31.25,37.8125,0.0,12.47,4.371958,...,2.820182,14.0000,17.75,0.00,10.00,3.338125,3.08,1.655016,2.1600,4.3350
86,23.50,46.25,34.873229,35.25,4.526997,31.75,38.2500,0.0,14.82,4.380583,...,3.127813,13.7500,18.00,0.00,9.51,3.424646,3.27,1.689198,2.1700,4.5000


#### iii. Standard Deviation

In [11]:
feature_columns = ['min_avg_rss12', 'max_avg_rss12', 'mean_avg_rss12', 'median_avg_rss12', 'std_avg_rss12',
                   'Q1_avg_rss12', 'Q3_avg_rss12', 'min_var_rss12', 'max_var_rss12', 'mean_var_rss12',
                   'median_var_rss12', 'std_var_rss12', 'Q1_var_rss12', 'Q3_var_rss12', 'min_avg_rss13',
                   'max_avg_rss13', 'mean_avg_rss13', 'median_avg_rss13', 'std_avg_rss13', 'Q1_avg_rss13',
                   'Q3_avg_rss13', 'min_var_rss13', 'max_var_rss13', 'mean_var_rss13', 'median_var_rss13',
                   'std_var_rss13', 'Q1_var_rss13', 'Q3_var_rss13', 'min_avg_rss23', 'max_avg_rss23',
                   'mean_avg_rss23', 'median_avg_rss23', 'std_avg_rss23', 'Q1_avg_rss23', 'Q3_avg_rss23',
                   'min_var_rss23', 'max_var_rss23', 'mean_var_rss23', 'median_var_rss23', 'std_var_rss23',
                   'Q1_var_rss23', 'Q3_var_rss23']

results_list = []

for feature in feature_columns:
    data = features_df[feature].dropna().values
    bootstrap_result = bs.bootstrap(data, stat_func=bs_stats.std, alpha=0.10)
    
    results_list.append({
        'Feature': feature,
        'Standard_Deviation': np.std(data),
        'CI_Lower': bootstrap_result.lower_bound,
        'CI_Upper': bootstrap_result.upper_bound
    })

results_df = pd.DataFrame(results_list)

results_df.sort_values(by='Feature')

,Feature,Standard_Deviation,CI_Lower,CI_Upper
5,Q1_avg_rss12,6.118526,5.634705,6.705613
19,Q1_avg_rss13,4.196608,3.703207,4.779061
33,Q1_avg_rss23,6.061727,4.955891,7.387159
12,Q1_var_rss12,0.940994,0.852951,1.058739
26,Q1_var_rss13,0.838813,0.791764,0.907221
40,Q1_var_rss23,0.754261,0.705612,0.823610
6,Q3_avg_rss12,5.109643,4.411644,5.913426
20,Q3_avg_rss13,4.147858,3.634768,4.780184
34,Q3_avg_rss23,5.500200,4.507051,6.651369
13,Q3_var_rss12,2.113157,1.944978,2.348697


#### iv. Select Features

mean, median, and standard deviation<br>
It feels that overall these three features have the lowest standard deviation within.

## 2. ISLR 3.7.4

### (a) Linear Train

In this case, we could expect the training RSS for cubic regression to be lower than that of linear regression. Cubic regression is a more flexible model and may be overfitting the data, which can lead to a lower training RSS.

### (b) Linear Test

For test RSS, however, we would expect the opposite result. Since the true relationship between X and Y is linear, we could expect the test RSS for linear regression to be lower than that of cubic regression. 

### (c) Not Linear Train

In this scenario, we would expect cubic regression to have a lower training RSS than linear regression. Cubic regression models are more flexible and can better adapt to non-linear relationships.

### (d) Not Linear Testing

There is not enough information to tell. It depends on how exactly the true relationship is between X and Y.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 